In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

In [2]:
from phenum.symmetry import get_lattice_pointGroup

In [3]:
import numpy as np
from numpy.linalg import inv, norm
from itertools import product
from copy import deepcopy

from BZI.symmetry import (find_orbitals, make_ptvecs,
                          make_rptvecs, reduce_kpoint_list,
                          find_orbits, find_orbitals, 
                          brillouin_zone_mapper)
from BZI.sampling import make_cell_points, sphere_pts
from BZI.plots import plot_mesh
from phenum.vector_utils import _minkowski_reduce_basis

In [4]:
from BZI.pseudopots import Al_PP

In [5]:
lat_consts = [2*np.pi]*3
lat_angles = [np.pi/2]*3
lat_center = 'body'
lat_vecs = make_ptvecs(lat_center, lat_consts, lat_angles)
rlat_vecs = make_rptvecs(lat_vecs)

HNF = [[2,0,0],[0,2,0],[0,0,2]]
grid_vecs = np.dot(rlat_vecs, inv(HNF))

grid_offset = [0.0, 0.0, 0.0]
# offset = np.dot(inv(grid_vecs), np.dot(rlat_vecs, grid_offset))
grid = np.array(make_cell_points(rlat_vecs, grid_vecs, grid_offset))

In [6]:
print("shift = (/ " + "_dp, ".join(map(str, np.round(grid_offset, 4))) + "_dp" + " /)")

# Print the reciprocal lattice vectors in a format that is 
# easy to copy and paste into Fortran code.
for i, r in enumerate(rlat_vecs):
    if i == 0:
        print("R = transpose(reshape((/ " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp, &")
    elif i == 1:
        print("                          " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp, &")
    else: 
        print("                          " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp /)," + 
             "(/3,3/)))")
        
# Print the reciprocal lattice vectors in a format that is 
# easy to copy and paste into Fortran code.
for i, r in enumerate(HNF):
    if i == 0:
        print("H = transpose(reshape((/ " + 
              ", ".join(map(str, np.round(r, 12))) + ", &")
    elif i == 1:
        print("                          " + 
              ", ".join(map(str, np.round(r, 12))) + ", &")
    else:
        print("                          " + 
              ", ".join(map(str, np.round(r, 12))) + " /)," + 
             "(/3,3/)))")

shift = (/ 0.0_dp, 0.0_dp, 0.0_dp /)
R = transpose(reshape((/ -0.0_dp, 1.0_dp, 1.0_dp, &
                          1.0_dp, 0.0_dp, 1.0_dp, &
                          1.0_dp, 1.0_dp, 0.0_dp /),(/3,3/)))
H = transpose(reshape((/ 2, 0, 0, &
                          0, 2, 0, &
                          0, 0, 2 /),(/3,3/)))


In [8]:
brillouin_zone_mapper(grid, rlat_vecs, grid_vecs, grid_offset)

reduced_grid
 [[ 0.   0.   0. ]
 [ 0.5  0.5  0. ]
 [ 1.   1.   1. ]]
mink basis
 [[ 1.  0. -0.]
 [ 0.  1.  1.]
 [ 1. -1.  1.]]
M
 [[-0.5  0.5  0.5]
 [ 1.   0.   0. ]
 [-0.5  1.5 -0.5]]
[ 0.   0.5  1. ]
[ 0.75  0.    0.25]
transformed grid
 [[ 0.   0.   0. ]
 [ 0.   0.5  0.5]
 [ 0.5  1.   0.5]]


(array([[ 0. ,  0. ,  0. ],
        [ 0. ,  0.5,  0.5],
        [-0.5,  0. ,  0.5]]), [1, 6, 1])